## № задача 1 

анализ эффективности рекламной кампании с учётом платформ
Компания запустила A/B-тест новой рекламной кампании. Пользователи случайным образом разделены на две группы:

- A — контрольная группа, видит старую кампанию;

- B — тестовая группа, видит новую кампанию.

Платформы пользователей — mobile и desktop. Известно, что поведение пользователей и их кликабельность (CTR) может отличаться в зависимости от платформы, поэтому важно провести стратифицированный анализ.

In [1]:
import pandas as pd
import numpy as np

In [3]:
# Пример данных
np.random.seed(42)
n = 1000
df = pd.DataFrame({
    "user_id": range(n),
    "platform": np.random.choice(["mobile", "desktop"], size=n, p=[0.6, 0.4]),
    "group": np.random.choice(["A", "B"], size=n),
})

# Симулируем метрику: выше на desktop
df["ctr"] = np.where(df["platform"] == "desktop",
                     np.random.normal(0.06, 0.01, n),
                     np.random.normal(0.03, 0.01, n))

# Стратификация: считаем среднее CTR по платформам и группам
result = df.groupby(["platform", "group"])["ctr"].mean().unstack()
print(result)

group            A         B
platform                    
desktop   0.060662  0.060753
mobile    0.029680  0.029867


 Разница между группами в каждой страте минимальна — скорее всего, тест не дал результата.

In [4]:
# Взвешенное среднее для группы A
weights = df["platform"].value_counts(normalize=True).sort_index()
weighted_mean_A = (result["A"] * weights).sum()
print("Weighted mean A:", weighted_mean_A)

Weighted mean A: 0.04167006489395897


In [5]:
from scipy.stats import ttest_ind

# Пример: метрика по группам внутри страты 'mobile'
a_mobile = df[(df['group'] == 'A') & (df['platform'] == 'mobile')]["ctr"]
b_mobile = df[(df['group'] == 'B') & (df['platform'] == 'mobile')]["ctr"]

t_stat, p_val = ttest_ind(a_mobile, b_mobile)
print("p-value для страты mobile:", p_val)

p-value для страты mobile: 0.8134668041040406


* p-value > 0.05 → нет статистически значимой разницы между A и B внутри mobile

* Разница между 2.97% и 2.99% — незначима

Общий вывод:
* Различий между группами A и B не наблюдается ни в одной страте
* Стратификация (по платформам) помогает отдельно проверить эффект A/B-теста на разных подгруппах пользователей
* Взвешенные средние показывают общее поведение метрики с учётом реальных пропорций платформ



## № задача 2 

стратифицированная выборка для A/B-теста

Условие:
Нужно разбить датафрейм на 2 равные по размеру и стратифицированные группы: пилотную и контрольную. Стратификация — по заданным колонкам (например, ['platform', 'gender']), с учётом весов страт (если не заданы — пропорционально числу объектов). Разбивка должна быть воспроизводимой при фиксированном seed и рандомной при его отсутствии.

In [6]:
def select_stratified_groups(data, strat_columns, group_size, weights=None, seed=None):
    np.random.seed(seed)
    
    # Определяем страты
    data = data.copy()
    data['stratum'] = data[strat_columns].apply(lambda row: tuple(row), axis=1)

    # Вычисляем веса страт, если не заданы
    if weights is None:
        stratum_counts = data['stratum'].value_counts(normalize=True)
        weights = stratum_counts.to_dict()

    # Подсчитываем, сколько объектов взять из каждой страты
    stratum_sizes = {
        stratum: int(round(group_size * weight))
        for stratum, weight in weights.items()
    }

    pilot_group = []
    control_group = []

    for stratum, size in stratum_sizes.items():
        group = data[data['stratum'] == stratum]
        if len(group) < 2 * size:
            raise ValueError(f"Недостаточно объектов в страте {stratum} для выборки {2*size}.")

        # Случайная перестановка
        shuffled = group.sample(frac=1, random_state=None)
        pilot_group.append(shuffled.iloc[:size])
        control_group.append(shuffled.iloc[size:2*size])

    pilot_df = pd.concat(pilot_group).drop(columns=['stratum']).reset_index(drop=True)
    control_df = pd.concat(control_group).drop(columns=['stratum']).reset_index(drop=True)

    return pilot_df, control_df

In [7]:
# Пример датафрейма
df = pd.DataFrame({
    "user_id": range(1000),
    "platform": np.random.choice(["ios", "android"], 1000, p=[0.4, 0.6]),
    "gender": np.random.choice(["male", "female"], 1000)
})
df.head()

,user_id,platform,gender
0,0,android,female
1,1,ios,male
2,2,ios,female
3,3,ios,female
4,4,ios,male


In [8]:
# Запуск функции
pilot, control = select_stratified_groups(df, strat_columns=["platform", "gender"], group_size=300, seed=42)

# Проверим распределение
print("Pilot group strat dist:")
print(pilot.groupby(["platform", "gender"]).size() / len(pilot))

print("\nControl group strat dist:")
print(control.groupby(["platform", "gender"]).size() / len(control))

Pilot group strat dist:
platform  gender
android   female    0.290970
          male      0.294314
ios       female    0.220736
          male      0.193980
dtype: float64

Control group strat dist:
platform  gender
android   female    0.290970
          male      0.294314
ios       female    0.220736
          male      0.193980
dtype: float64



| platform | gender | доля  |
| -------- | ------ | ----- |
| android  | female | 29.1% |
| android  | male   | 29.4% |
| ios      | female | 22.1% |
| ios      | male   | 19.4% |


Абсолютно такие же доли, как в пилотной группе.

* Это результат стратифицированного семплирования по колонкам `platform` и `gender`.
* **Доли подгрупп (страт)** между пилотной и контрольной группами **одинаковы**, что и требовалось.
* Например, 29.1% пользователей в обеих группах — это **женщины на Android**, 22.1% — **женщины на iOS**, и так далее.

---

* Это **обеспечивает баланс** между группами A и B по ключевым признакам.
* Такие группы позволяют провести **более точный и справедливый A/B‑тест**, снизив влияние дисперсии и случайных перекосов.
* Без этого — различия между группами могли бы быть связаны, скажем, с тем, что в одной группе больше женщин на Android, чем в другой, а не с самим эффектом тестируемой фичи.


> Мы убедились, что обе группы **абсолютно одинаковы по составу** — значит, различия в метриках (например, CTR, CR и т.п.) будут обусловлены **реальным эффектом**, а не случайным смещением данных. Это ключ к корректной интерпретации A/B‑теста.
